In [70]:
############################## can we decide between task related and MW aware and MW unaware ? ############################


## Dependencies

# Import package/module for data
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer



# Import modules for feature engineering and modelling
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GroupKFold

# pipeline
#from sklearn.pipeline import Pipeline, FeatureUnion
from imblearn.pipeline import Pipeline

# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# cross validation
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GroupKFold

# balancing
from imblearn.over_sampling import SMOTE

#accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from IPython.display import display

from collections import Counter


In [71]:
read_path_awareness = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\features_with_label_awareness.csv"
read_path = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\features_with_labels.csv"
df = pd.read_csv(read_path)

#print(df.columns.tolist())

In [72]:
#df["Awareness_all"].unique()
df["Awareness_all"].value_counts()

task-related    631
aware           249
unaware         147
Name: Awareness_all, dtype: int64

In [73]:
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df["Awareness_all_new"] = labelencoder.fit_transform(df["Awareness_all"])
df["Awareness_all_new"].unique()

array([1, 0, 2])

In [74]:
df["Awareness_all_new"].value_counts()
# 1 = task related, 0 aware 2 unaware

1    631
0    249
2    147
Name: Awareness_all_new, dtype: int64

In [75]:
# ['task-related' == 1 , 'aware' == 2, 'unaware' == 3 , nan == delete the row]
def get_X_y(train):
    FEATURES = [
        'Fixation Duration Mean [ms]', 'Fixation Duration Max [ms]', 'Fixation Duration Min [ms]', 'Fixation Duration Median [ms]', 'Fixation Duration Std [ms]', 'Fixation Duration Skew [ms]', 'Fixation Duration Quantil 25 [ms]', 'Fixation Duration Quantil 75 [ms]',
        'Saccade Duration Mean [ms]', 'Saccade Duration Max [ms]', 'Saccade Duration Min [ms]', 'Saccade Duration Median [ms]', 'Saccade Duration Std [ms]', 'Saccade Duration Skew [ms]', 'Saccade Duration Quantil 25 [ms]', 'Saccade Duration Quantil 75 [ms]', 
        'Blink Duration Mean [ms]', 'Blink Duration Max [ms]', 'Blink Duration Min [ms]', 'Blink Duration Median [ms]', 'Blink Duration Std [ms]', 'Blink Duration Skew [ms]', 'Blink Duration Quantil 25 [ms]', 'Blink Duration Quantil 75 [ms]', 'Fixation Duration Kurtosis [ms]',
        'Saccade Duration Kurtosis [ms]',
        'Blink Duration Kurtosis [ms]', 
        'Fixation Saccade Ratio Mean', 'Fixation Saccade Ratio Max', 'Fixation Saccade Ratio Min', 'Fixation Saccade Ratio Median', 'Fixation Saccade Ratio Std', 'Fixation Saccade Ratio Skew', 'Fixation Saccade Ratio Kurtosis', 
        'Fixation Number', 'Blink Number', 
        'Fixation Dispersion X Mean [px]', 'Fixation Dispersion X Max [px]', 'Fixation Dispersion X Min [px]', 'Fixation Dispersion X Median [px]', 'Fixation Dispersion X Std [px]', 'Fixation Dispersion X Skew [px]', 'Fixation Dispersion X Quantil 25 [px]', 'Fixation Dispersion X Quantil 75 [px]', 
        'Fixation Dispersion Y Mean [px]', 'Fixation Dispersion Y Max [px]', 'Fixation Dispersion Y Min [px]', 'Fixation Dispersion Y Median [px]', 'Fixation Dispersion Y Std [px]', 'Fixation Dispersion Y Skew [px]', 'Fixation Dispersion Y Quantil 25 [px]', 'Fixation Dispersion Y Quantil 75 [px]', 'Fixation Dispersion X Kurtosis [px]', 'Fixation Dispersion Y Kurtosis [px]', 
        'Saccade Amplitude Mean [°]', 'Saccade Amplitude Max [°]', 'Saccade Amplitude Min [°]', 'Saccade Amplitude Median [°]', 'Saccade Amplitude Std [°]', 'Saccade Amplitude Skew [°]', 'Saccade Amplitude Quantil 25 [°]', 'Saccade Amplitude Quantil 75 [°]', 'Saccade Amplitude Kurtosis [°]',
        'Saccade Acceleration Average [°/s²] Mean', 'Saccade Acceleration Average [°/s²] Max', 'Saccade Acceleration Average [°/s²] Min', 'Saccade Acceleration Average [°/s²] Median', 'Saccade Acceleration Average [°/s²] Std', 'Saccade Acceleration Average [°/s²] Skew]', 'Saccade Acceleration Average [°/s²] Quantil 25]', 'Saccade Acceleration Average [°/s²] Quantil 75]',
        'Saccade Acceleration Peak [°/s²] Mean', 'Saccade Acceleration Peak [°/s²] Max', 'Saccade Acceleration Peak [°/s²] Min', 'Saccade Acceleration Peak [°/s²] Median', 'Saccade Acceleration Peak [°/s²] Std', 'Saccade Acceleration Peak [°/s²] Skew]', 'Saccade Acceleration Peak [°/s²] Quantil 25]', 'Saccade Acceleration Peak [°/s²] Quantil 75]', 'Saccade Deceleration Peak [°/s²] Mean', 
        'Saccade Deceleration Peak [°/s²] Max', 'Saccade Deceleration Peak [°/s²] Min', 'Saccade Deceleration Peak [°/s²] Median', 'Saccade Deceleration Peak [°/s²] Std', 'Saccade Deceleration Peak [°/s²] Skew]', 'Saccade Deceleration Peak [°/s²] Quantil 25]', 'Saccade Deceleration Peak [°/s²] Quantil 75]', 
        'Saccade Velocity Average [°/s²] Mean', 'Saccade Velocity Average [°/s²] Max', 'Saccade Velocity Average [°/s²] Min', 'Saccade Velocity Average [°/s²] Median', 'Saccade Velocity Average [°/s²] Std', 'Saccade Velocity Average [°/s²] Skew]', 'Saccade Velocity Average [°/s²] Quantil 25]', 'Saccade Velocity Average [°/s²] Quantil 75]', 
        'Saccade Velocity Peak [°/s²] Mean', 'Saccade Velocity Peak [°/s²] Max', 'Saccade Velocity Peak [°/s²] Min', 'Saccade Velocity Peak [°/s²] Median', 'Saccade Velocity Peak [°/s²] Std', 'Saccade Velocity Peak [°/s²] Skew]', 'Saccade Velocity Peak [°/s²] Quantil 25]', 'Saccade Velocity Peak [°/s²] Quantil 75]', 
        'Saccade Velocity Peak [%] Mean', 'Saccade Velocity Peak [%] Max', 'Saccade Velocity Peak [%] Min', 'Saccade Velocity Peak [%] Median', 'Saccade Velocity Peak [%] Std', 'Saccade Velocity Peak [%] Skew]', 'Saccade Velocity Peak [%] Quantil 25]', 'Saccade Velocity Peak [%] Quantil 75]', 
        'Saccade Acceleration Average [°/s²] Kurtosis', 'Saccade Acceleration Peak [°/s²] Kurtosis', 'Saccade Deceleration Peak [°/s²] Kurtosis', 'Saccade Velocity Average [°/s²] Kurtosis', 'Saccade Velocity Peak [°/s²] Kurtosis', 'Saccade Velocity Peak [%] Kurtosis', 
        'Saccade Length Mean [px]', 'Saccade Length Max [px]', 'Saccade Length Min [px]', 'Saccade Length Median [px]', 'Saccade Length Std [px]', 'Saccade Length Skew [px]]', 'Saccade Length Quantil 25 [px]]', 'Saccade Length Quantil 75 [px]]', 'Saccade Length Kurtosis [px]', 
        'Fixation Average Pupil Diameter [mm] Mean', 'Fixation Average Pupil Diameter [mm] Max', 'Fixation Average Pupil Diameter [mm] Min', 'Fixation Average Pupil Diameter [mm] Median', 'Fixation Average Pupil Diameter [mm] Std', 'Fixation Average Pupil Diameter [mm] Skew', 'Fixation Average Pupil Diameter [mm] Quantil25', 'Fixation Average Pupil Diameter [mm] Quantil75',
        'Fixation Average Pupil Diameter [mm] Kurtosis', 
        'Veregence Angles Mean [rad]', 'Veregence Angles Std [rad]', 
        'Pupil Distance Mean [px]', 'Pupil Distance Std [px]'
    ]

    GROUPS = "Participant"

    TARGET = "Awareness_all_new"

    X = train[FEATURES]
    y = train[TARGET]
    groups = train[GROUPS]
    return X, y, groups

In [76]:
X, y, groups = get_X_y(df)

In [77]:
##################### preprocessing with pipleline #####################
imputer =  SimpleImputer(fill_value='missing')
scaler = StandardScaler()
over = SMOTE(random_state= 27) #sampling_strategy=0.1

model = xgb.XGBClassifier(objective="objective=multi:softmax", random_state=42)
#model = GaussianNB()

#model = RandomForestClassifier(random_state=0)
#model = SVC(kernel="rbf", C=0.025, probability=True)

steps = [('imputer', imputer), ('scaler',scaler),('over', over), ('model', model)] #GaussianNB())## LinearSVC()) 
pipe = Pipeline(steps=steps)

In [78]:
##################### prediction without baseline #####################
import random
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0
accurancy_scores = []
f1_scores = []
f1_scores ={"0": [], "1": [], "2": []}
precision_scores = []
recall_scores = []

### stratifies group k fold
for train_index, test_index in sgk.split(X, y, groups):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # Fit Model on Train
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    #accurancy_score = accuracy_score(y_test, y_pred)

    labels = [0, 1, 2]
    f1 = f1_score(y_test, y_pred, average=None, labels=labels)
    f1_scores["0"].append(f1[0])
    f1_scores["1"].append(f1[1])
    f1_scores["2"].append(f1[2])
    print(f1)

    # precision = precision_score(y_test, y_pred, average=None)
    # recall = recall_score(y_test, y_pred, average=None)

    #accurancy_scores.append(accurancy_score)
   # print(pd.DataFrame(confusion_matrix(y_test, y_pred)))
    # f1_scores.append(f1)
    # precision_scores.append(precision)
    # recall_scores.append(recall)

    #print(f"Accuracy for fold {fold}: ", accurancy_score)
    print(f"f1 with labels  score for fold {fold}: ", {label:score for label,score in zip(labels, f1)})
    # print(f"recall for fold {fold}: ", recall)
    # print(f"precision for fold {fold}: ", precision)
    
    fold += 1

averages = [(k, sum(v)/len(v)) for k, v in f1_scores.items()]
for k, v in averages:
    print("label", k, f"\n{v}")

# mean_score = np.mean(accurancy_scores)
# mean_f1 = np.mean(f1_scores)
# mean_recall = np.mean(recall_scores)
# mean_precision = np.mean(precision_scores)

# #print(f'Our mean fold accurancy_scores is {mean_score:0.4f}')
# print(f'Our mean fold f1 score is {mean_f1:0.4f}')
# print(f'Our mean fold recall is {mean_recall:0.4f}')
# print(f'Our mean fold precisionis {mean_precision:0.4f}')


[0.11627907 0.62876254 0.06349206]
f1 with labels  score for fold 0:  {0: 0.11627906976744186, 1: 0.62876254180602, 2: 0.06349206349206349}
[0.28235294 0.75       0.        ]
f1 with labels  score for fold 1:  {0: 0.2823529411764706, 1: 0.75, 2: 0.0}
[0.41322314 0.71365639 0.23809524]
f1 with labels  score for fold 2:  {0: 0.4132231404958678, 1: 0.7136563876651981, 2: 0.2380952380952381}
[0.3960396  0.76156584 0.        ]
f1 with labels  score for fold 3:  {0: 0.396039603960396, 1: 0.7615658362989324, 2: 0.0}
[0.35294118 0.6745098  0.10526316]
f1 with labels  score for fold 4:  {0: 0.35294117647058826, 1: 0.6745098039215687, 2: 0.10526315789473685}
label 0 
0.3121671863741529
label 1 
0.7056989139383438
label 2 
0.08137009189640769


In [85]:
##################### prediction with baseline #####################
# task-related    631
# aware           249
# unaware         147


import random
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0
accurancy_scores = []
f1_scores = []
f1_scores ={"0": [], "1": [], "2": []}
precision_scores = []
recall_scores = []

### stratifies group k fold
for train_index, test_index in sgk.split(X, y, groups):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # Fit Model on Train
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    # create baseline
    baseline = np.ones(len(y_pred))
    mw__aware_size = 24.24537/100  * len(y_pred)
    mw__unaware_size = 14.31353/100  * len(y_pred)

    baseline[:int(mw__aware_size)] = 0
    baseline[int(mw__aware_size): int(mw__unaware_size)] = 2
    np.random.shuffle(baseline)
    baseline = baseline.astype(int)

    #accurancy_score = accuracy_score(y_test, y_pred)

    labels = [0, 1, 2]
    f1 = f1_score(y_test, baseline, average=None, labels=labels)
    f1_scores["0"].append(f1[0])
    f1_scores["1"].append(f1[1])
    f1_scores["2"].append(f1[2])
    print(f1)

    # precision = precision_score(y_test, y_pred, average=None)
    # recall = recall_score(y_test, y_pred, average=None)

    #accurancy_scores.append(accurancy_score)
   # print(pd.DataFrame(confusion_matrix(y_test, y_pred)))
    # f1_scores.append(f1)
    # precision_scores.append(precision)
    # recall_scores.append(recall)

    #print(f"Accuracy for fold {fold}: ", accurancy_score)
    print(f"f1 with labels  score for fold {fold}: ", {label:score for label,score in zip(labels, f1)})
    # print(f"recall for fold {fold}: ", recall)
    # print(f"precision for fold {fold}: ", precision)
    
    fold += 1

averages = [(k, sum(v)/len(v)) for k, v in f1_scores.items()]
for k, v in averages:
    print("label", k, f"\n{v}")

# mean_score = np.mean(accurancy_scores)
# mean_f1 = np.mean(f1_scores)
# mean_recall = np.mean(recall_scores)
# mean_precision = np.mean(precision_scores)

# #print(f'Our mean fold accurancy_scores is {mean_score:0.4f}')
# print(f'Our mean fold f1 score is {mean_f1:0.4f}')
# print(f'Our mean fold recall is {mean_recall:0.4f}')
# print(f'Our mean fold precisionis {mean_precision:0.4f}')


[0.22222222 0.60135135 0.        ]
f1 with labels  score for fold 0:  {0: 0.2222222222222222, 1: 0.6013513513513513, 2: 0.0}
[0.21686747 0.71942446 0.        ]
f1 with labels  score for fold 1:  {0: 0.21686746987951808, 1: 0.7194244604316546, 2: 0.0}
[0.23157895 0.65917603 0.        ]
f1 with labels  score for fold 2:  {0: 0.23157894736842108, 1: 0.6591760299625468, 2: 0.0}
[0.14893617 0.7114094  0.        ]
f1 with labels  score for fold 3:  {0: 0.14893617021276598, 1: 0.7114093959731544, 2: 0.0}
[0.29906542 0.66911765 0.        ]
f1 with labels  score for fold 4:  {0: 0.2990654205607476, 1: 0.6691176470588235, 2: 0.0}
label 0 
0.223734046048735
label 1 
0.672095776955506
label 2 
0.0


In [83]:
# create baseline 
        # 38.55891% von y_pred should be MW, MW = 0
# # task-related    631 | 1
# aware           249 | 0
# unaware         147 | 2
# 1027
baseline = np.ones(len(y_pred))
mw__aware_size = 24.24537/100  * len(y_pred)
mw__unaware_size = 14.31353/100  * len(y_pred)

baseline[:int(mw__aware_size)] = 0
baseline[int(mw__aware_size): int(mw__unaware_size)] = 2
np.random.shuffle(baseline)
baseline = baseline.astype(int)